In [1]:
import numpy as np
from numpy import asarray
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from keras.models import Sequential
from keras.layers import LSTM, Dropout
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Flatten
from tqdm import tqdm
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from xgboost import XGBRegressor

In [23]:
from sklearn.tree import ExtraTreeRegressor

In [3]:
df = pd.read_csv('Dasatest_N0.csv')
df['Month'] = pd.to_datetime(df.Month , format = '%d/%m/%Y')
data = df.drop(['Month'], axis=1)
data.index = df.Month
cols = data.columns

In [4]:
pred = np.zeros([15,3915])

In [5]:
# preparing independent and dependent features
def prepare_data(timeseries_data, n_features):
	X, y =[],[]
	for i in range(len(timeseries_data)):
		# find the end of this pattern
		end_ix = i + n_features
		# check if we are beyond the sequence
		if end_ix > len(timeseries_data)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [6]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
# 	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols = list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
	# put it all together
	agg = concat(cols, axis=1)
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg.values  

In [7]:
def model_create(X, y):
    model = XGBRegressor(objective='reg:squarederror', n_estimators=10)
    model.fit(X, y)
    return model

In [8]:
def model_create_2(X, y):
    model = RandomForestRegressor(n_estimators=5)
    model.fit(X, y)
    return model

In [27]:
def model_create_3(X, y):
    model = DecisionTreeRegressor()
    model.fit(X, y)
    return model

In [26]:
def model_create_4(X, y):
    model = ExtraTreeRegressor()
    model.fit(X, y)
    return model

In [10]:
scaler = MinMaxScaler()

In [11]:
def model_predict(model,X_input,n_steps,n_steps_out):   
    x_input = X_input
#     print(x_input)
    temp_input = list(x_input)
#     print(len(temp_input))
    lst_output=[]
    i = 0
    while(i < n_steps_out):
        
        if(len(temp_input)>n_steps):
            x_input=np.array(temp_input[1:])
#             print("{} day input {}".format(i,x_input))
            #print(x_input)
            x_input = x_input.reshape(n_steps)
            #print(x_input)
            yhat = model.predict(asarray([x_input]))
#             print("{} day output {}".format(i,yhat))
            temp_input.append(yhat[0])
            temp_input=temp_input[1:]
            #print(temp_input)
            lst_output.append(yhat[0])
            i=i+1
        else:
            x_input = x_input.reshape(n_steps)
            yhat = model.predict(asarray([x_input]))
#             print(yhat[0])
            temp_input.append(yhat[0])
            lst_output.append(yhat[0])
            i=i+1

#     print(lst_output)
    return(np.array(lst_output))

In [12]:
def plot_graph(X,pred):
    time = np.arange(87)
    plt.plot(time[60:72],X)
    plt.plot(time[60:72],pred)
    plt.show()

In [13]:
prediction = np.zeros([15,3915])
prediction1 = np.zeros([15,3915])
sum1 = 0 

In [14]:
def model_sarakam(n_in,train_real,rem):
    train = series_to_supervised(train_real, n_in=n_in)
    # split into input and output columns
    trainX, trainy = train[:, :-1], train[:, -1]
#     print(trainX.shape)
    model = model_create(trainX, rem[-len(trainX):])
    X_input = data_total[:n_in].reshape((n_in))
    n_steps_out = len(trainX)
    pred = model_predict(model,X_input,n_in,n_steps_out)
    pred = np.array(pred)
    return pred,model

In [15]:
def model_sarakam_2(n_in,train_real,rem):
    train = series_to_supervised(train_real, n_in=n_in)
    # split into input and output columns
    trainX, trainy = train[:, :-1], train[:, -1]
#     print(trainX.shape)
    model = model_create_2(trainX, rem[-len(trainX):])
    X_input = data_total[:n_in].reshape((n_in))
    n_steps_out = len(trainX)
    pred = model_predict(model,X_input,n_in,n_steps_out)
    pred = np.array(pred)
    return pred,model

In [16]:
sum1 = 0
for i in tqdm(range(3915)):
    data_total = np.array(data.iloc[:,i])
    train_real = data_total
#     valid = data_total[60:]
    
    n_in_1 = 6
    pred1, model1 = model_sarakam(n_in_1,train_real,train_real)
    rem1 = train_real[-len(pred1):] - pred1
    
    n_in_2 = 6
    pred2, model2 = model_sarakam_2(n_in_2,train_real,rem1)
    rem2 = rem1[-len(pred2):] - pred2
    
    n_in_3 = 12
    pred3, model3 = model_sarakam(n_in_3,train_real,rem2)
    rem3 = rem2[-len(pred3):] - pred3
    
    n_in_4 = 12
    pred4, model4 = model_sarakam_2(n_in_4,train_real,rem3)
    rem4 = rem3[-len(pred4):] - pred4
    
    n_in_5 = 18
    pred5, model5 = model_sarakam(n_in_5,train_real,rem4)
    rem5 = rem4[-len(pred5):] - pred5
    
    n_in_6 = 18
    pred6, model6 = model_sarakam_2(n_in_6,train_real,rem5)
    rem6 = rem5[-len(pred6):] - pred6
    
    n_in_7 = 24
    pred7, model7 = model_sarakam(n_in_7,train_real,rem6)
    rem7 = rem6[-len(pred7):] - pred7
    
    n_in_8 = 24
    pred8, model8 = model_sarakam_2(n_in_8,train_real,rem7)
    rem8 = rem7[-len(pred8):] - pred8
    
    n_in_9 = 30
    pred9, model9 = model_sarakam(n_in_9,train_real,rem8)
    rem9 = rem8[-len(pred9):] - pred9
    
    n_in_10 = 30
    pred10, model10 = model_sarakam_2(n_in_10,train_real,rem9)
    rem10 = rem9[-len(pred10):] - pred10
    
    n_in_11 = 36
    pred11, model11 = model_sarakam(n_in_11,train_real,rem10)
    rem11 = rem10[-len(pred11):] - pred11
    
    n_in_12 = 36
    pred12, model12 = model_sarakam_2(n_in_12,train_real,rem11)
    rem12 = rem11[-len(pred12):] - pred12
    
    n_steps_out = 15
    
    X_input_12 = train_real[-n_in_12:]
    fpred_12 = model_predict(model12,X_input_12,n_in_12,n_steps_out)
    
    X_input_11 = train_real[-n_in_11:]
    fpred_11 = model_predict(model11,X_input_11,n_in_11,n_steps_out)
    
    X_input_10 = train_real[-n_in_10:]
    fpred_10 = model_predict(model10,X_input_10,n_in_10,n_steps_out)
    
    X_input_9 = train_real[-n_in_9:]
    fpred_9 = model_predict(model9,X_input_9,n_in_9,n_steps_out)
    
    X_input_8 = train_real[-n_in_8:]
    fpred_8= model_predict(model8,X_input_8,n_in_8,n_steps_out)
    
    X_input_7 = train_real[-n_in_7:]
    fpred_7 = model_predict(model7,X_input_7,n_in_7,n_steps_out)
    
    X_input_6 = train_real[-n_in_6:]
    fpred_6 = model_predict(model6,X_input_6,n_in_6,n_steps_out)
    
    X_input_5 = train_real[-n_in_5:]
    fpred_5 = model_predict(model5,X_input_5,n_in_5,n_steps_out)
    
    X_input_4 = train_real[-n_in_4:]
    fpred_4 = model_predict(model4,X_input_4,n_in_4,n_steps_out)
    
    X_input_3 = train_real[-n_in_3:]
    fpred_3 = model_predict(model3,X_input_3,n_in_3,n_steps_out)
    
    X_input_2 = train_real[-n_in_2:]
    fpred_2 = model_predict(model2,X_input_2,n_in_2,n_steps_out)
    
    X_input_1 = train_real[-n_in_1:]
    fpred_1 = model_predict(model1,X_input_1,n_in_1,n_steps_out)
    
    fpred = (n_in_12*fpred_1+n_in_11*fpred_2+n_in_10*fpred_3+n_in_9*fpred_4+n_in_8*fpred_5+n_in_7*fpred_6+n_in_6*fpred_7+n_in_5*fpred_8+n_in_4*fpred_9+n_in_3*fpred_10+n_in_2*fpred_11+n_in_1*fpred_12)/(n_in_1+n_in_2+n_in_3+n_in_4+n_in_5+n_in_6+n_in_7+n_in_8+n_in_9+n_in_10+n_in_11+n_in_12)
    fpred1 = fpred_1 + fpred_2 + fpred_3 + fpred_4 + fpred_5 + fpred_6 + fpred_7 + fpred_8 + fpred_9 + fpred_10 + fpred_11 + fpred_12
    
    prediction[:,i] = fpred
    prediction1[:,i] = fpred1

100%|████████████████████████████████████████████████████████████████████████████| 3915/3915 [1:21:04<00:00,  1.24s/it]


In [17]:
print(prediction)
print(prediction1)


[[0.08424652 0.06031037 0.08882482 ... 0.55226058 0.75243898 0.28618298]
 [0.06108057 0.05472616 0.02395769 ... 0.49229259 0.44046285 0.21959249]
 [0.06715878 0.07530026 0.05038515 ... 0.52759575 0.25053214 0.22029721]
 ...
 [0.0227261  0.04634084 0.02581296 ... 0.27889707 0.27582175 0.0837275 ]
 [0.04195748 0.0247314  0.0280391  ... 0.17748775 0.28023465 0.11210028]
 [0.04339557 0.04780256 0.04594215 ... 0.14301951 0.17117921 0.08186308]]
[[3.81018011e-01 3.72491974e-01 6.29430640e-01 ... 6.96646975e+00
  6.63777511e+00 2.68435925e+00]
 [3.39186739e-01 4.03637314e-01 1.82893456e-01 ... 6.05665972e+00
  3.23816394e+00 2.19636318e+00]
 [6.25076493e-01 6.01351982e-01 6.39173357e-01 ... 6.46871642e+00
  8.47355899e-01 2.17659197e+00]
 ...
 [5.33763264e-03 4.05189965e-01 2.05085755e-01 ... 3.45340780e+00
  1.76942197e+00 2.12506820e-01]
 [1.68579387e-01 1.35067597e-01 2.33527320e-01 ... 1.61345793e+00
  1.64227875e+00 5.18953704e-01]
 [2.43068146e-01 4.13921130e-01 4.39898081e-01 ... 1.026

In [18]:
output = pd.DataFrame(prediction)
output.to_csv('submission_hybrid_mix_12.csv')
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [19]:
output = pd.DataFrame(prediction1)
output.to_csv('submission_hybrid_mix_turu_12.csv')
print("Your submission was successfully saved!")

Your submission was successfully saved!
